In [1]:
import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role
import time
import pandas as pd
import matplotlib.pyplot as plt
from helper import *
from datetime import timedelta

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# session, role, bucket
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

s3_bucket = sagemaker_session.default_bucket()
s3_prefix = 'MLEND-Capstone-Project'    

region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data_hyper_param".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output_hyper_param".format(s3_bucket, s3_prefix)

image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [3]:
stock_hyper_param_data = pd.read_csv('stock_data.csv',parse_dates=True, index_col=[0,1])
get_target_distribution(stock_hyper_param_data)

-1:  390748
 0:  389090
 1:  397485


In [4]:
tickers = get_sp500_tickers()

freq = 'D'

# we predict for 1 day
prediction_length = 1

# we use 50 days as context length, this is the number of state updates accomplished before making predictions
context_length = 50

end_training = pd.Timestamp('2018-12-31', freq=freq)

timeseries = []
    
for ID,ticker in list(enumerate(tickers)):
    ticker = stock_hyper_param_data.loc[(slice(None), ticker), :]
    if ticker.index[0][0]<end_training:
        timeseries.append(ticker)

In [42]:
tickers = []
for ts in timeseries:
    tickers.append(ts.index[1][1])
cat = {}
for ticker in enumerate(tickers):
    cat[ticker[1]] = ticker[0]


In [43]:
dynamic_feat = ['Adj Close','Volume','PC1','PC2','PC3','PC4','PC5','PC6']
training_data = [
    {
        "start": str(ts.index[0][0]),
        "target": ts['target'][ts.index[0][0]:end_training].tolist(), # We use -1, because pandas indexing includes the upper bound 
        "cat" : cat[ts.index[1][1]],
        "dynamic_feat": ts[dynamic_feat][ts.index[0][0]:end_training].values.T.tolist()
        
    }
    for ts in timeseries
]
print(len(training_data))

491


In [15]:
num_test_windows = 10

test_data = [
    {
        "start": str(ts.index[0][0]),
        "target": ts['target'][ts.index[0][0]:end_training + timedelta(days=(2*k * prediction_length))].tolist(),
        "cat" : cat[ts.index[1][1]], # input stock ticker id
        "dynamic_feat": ts[dynamic_feat][ts.index[0][0]:end_training + timedelta(days=(2*k * prediction_length))].values.T.tolist()
    }
    for k in range(1, num_test_windows + 1) 
    for ts in timeseries
]
print(len(test_data))



4910


In [16]:
%%time
write_json_dataset("train_hyper_param.json", training_data)
write_json_dataset("test_hyper_param.json", test_data)

CPU times: user 1min 9s, sys: 1.13 s, total: 1min 10s
Wall time: 1min 10s


In [17]:
copy_to_s3("train_hyper_param.json", s3_data_path + "/train/train.json", s3_bucket)
copy_to_s3("train_hyper_param.json", s3_data_path + "/test/test.json", s3_bucket)

Uploading file to s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data_hyper_param/train/train.json
Uploading file to s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data_hyper_param/test/test.json


In [18]:
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:10000] + "...")

{"start": "2010-03-16 00:00:00", "target": [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, -1, 1, 1, 0, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, -1, -1, 0, 1, 0, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 0, 0, -1, 0, 

In [19]:
estimator_hyper_param = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='MLEND-Capstone-Project',
    output_path=s3_output_path
)


In [20]:
#hyperparameters = {
    "time_freq": freq,
    "epochs": "100",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_cells": 50,
    "num_layers":2,
    "dropout_rate":0.1,
    "num_dynamic_feat": 'auto',
}
#estimator_hyper_param.set_hyperparameters(**hyperparameters)



In [21]:
#from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

#estimator_hyper_param_tuner = HyperparameterTuner(estimator = estimator_hyper_param, # The estimator object to use as the basis for the training jobs.
                                               objective_metric_name = 'test:RMSE', # The metric used to compare trained models.
                                               objective_type = 'Minimize', # Whether we wish to minimize or maximize the metric.
                                               max_jobs = 30, # The total number of models to train
                                               max_parallel_jobs = 2, # The number of models to train in parallel
                                               hyperparameter_ranges = {
                                                    'epochs': IntegerParameter(20, 100),
                                                    'context_length': IntegerParameter(10, 60),
                                                    'mini_batch_size': IntegerParameter(32, 256),
                                                    'learning_rate': ContinuousParameter("1E-5", "1E-3"),
                                                    'num_cells': IntegerParameter(30, 200),
                                                    'dropout_rate': ContinuousParameter(0,0.1),
                                                    'num_layers': IntegerParameter(1,3)
                                               })

In [22]:
%%time
data_channels = {
    "train": s3_data_path + "/train/train.json",
    "test": s3_data_path + "/test/test.json"
}

estimator_hyper_param_tuner.fit(inputs=data_channels, wait=True)

CPU times: user 8.84 ms, sys: 7.74 ms, total: 16.6 ms
Wall time: 224 ms


In [24]:
hyper_param_predictor = sagemaker.predictor.RealTimePredictor(endpoint='deep-ar-hyperparam-best')

In [47]:
get_stock_prediction('F', '2019-01-02',stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['F'])

/home/ec2-user/SageMaker/MLEND-Capstone-Project/helper.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['prediction'] = pred


,,Adj Close,Volume,PC1,PC2,PC3,PC4,PC5,PC6,target,prediction
Date,Ticker,,,,,,,,,,
2019-01-02,F,0.00187,0.055249,0.070807,0.258602,0.320558,0.455994,0.347852,0.282777,1,1


In [27]:
date_index = pd.read_csv('test_date_index.csv')
date_index = date_index.values.reshape(252).tolist()
acc = {}

In [ ]:
for ticker in tickers:
    acc[ticker] = get_prediction_accuracy(ticker, date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat[ticker])

/home/ec2-user/SageMaker/MLEND-Capstone-Project/helper.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['prediction'] = pred


In [32]:
#acc['A'] = get_prediction_accuracy('A', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['A'])
#acc['F'] = get_prediction_accuracy('F', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['F'])
#acc['GE'] = get_prediction_accuracy('GE', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['GE'])
#acc['DAL'] = get_prediction_accuracy('DAL', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['DAL'])
#acc['UAL'] = get_prediction_accuracy('UAL', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['UAL'])
#acc['ABC'] = get_prediction_accuracy('ABC', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['ABC'])
#acc['CAT'] = get_prediction_accuracy('CAT', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['CAT'])
#acc['DE'] = get_prediction_accuracy('DE', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['DE'])
#acc['D'] = get_prediction_accuracy('D', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['D'])
#acc['PEP'] = get_prediction_accuracy('PEP', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['PEP'])
#acc['IBM'] = get_prediction_accuracy('IBM', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['IBM'])
#acc['KMX'] = get_prediction_accuracy('KMX', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['KMX'])
#acc['YUM'] = get_prediction_accuracy('YUM', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['YUM'])
#acc['AIG'] = get_prediction_accuracy('AIG', date_index, stock_hyper_param_data,hyper_param_predictor, dynamic_feat, cat['AIG'])

In [ ]:
np.array(list(acc.values())).mean()

In [ ]:
acc